In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

# 좌석 선택 페이지로 이동한 후

# 좌석 선택 후 팝업 처리 호출
handle_alert()

# iframe 존재 확인 및 전환
try:
    # iframe이 로드될 때까지 대기
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
    )
    print("iframe 존재 확인")
    # iframe으로 전환
    driver.switch_to.frame(iframe)
    print("iframe 전환 성공")
except Exception as e:
    print(f"iframe 전환 오류: {e}")

# 좌석 클릭하기
try:
    seat_id = 't5900104'  # 좌석의 id
    seat_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, seat_id))
    )
    seat_element.click()
    print("좌석 클릭 완료")
except Exception as e:
    print(f"좌석 클릭 오류: {e}")


# 브라우저 유지
print("작업 완료, 브라우저를 닫지 않고 유지합니다.")
# driver.quit()  # 수동으로 브라우저 닫기 원할 시 사용

쿠키 로드 성공
예매하기 버튼 클릭 완료
새 창으로 제어를 전환했습니다.
날짜 선택 완료
회차 선택 완료
좌석 선택 완료
iframe 존재 확인
iframe 전환 성공
좌석 클릭 완료
작업 완료, 브라우저를 닫지 않고 유지합니다.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json
import time
import traceback
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException, TimeoutException

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용) 
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용) 
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

# 지속적인 좌석 모니터링 설정
monitoring_interval = 2  # 좌석 모니터링 주기 (초)
max_attempts = 100  # 최대 시도 횟수

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임으로 전환
            iframe_seat = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
            )
            driver.switch_to.frame(iframe_seat)
            print("Switched to iframe 'ifrmSeatFrame'")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 잠시 대기
                time.sleep(1)

                # 알림 창 처리 (필요한 경우)
                try:
                    WebDriverWait(driver, 3).until(EC.alert_is_present())
                    alert = driver.switch_to.alert
                    print(f"Alert text: {alert.text}")
                    alert.accept()
                    print("Alert accepted.")
                except TimeoutException:
                    print("No alert appeared after seat selection.")

                # 좌석 선택 후 기본 컨텐츠로 전환
                driver.switch_to.default_content()
                print("Switched to default content after clicking the seat")

                # 필요한 경우, 다시 프레임 전환
                # 여기서는 'ifrmSeatFrame' 프레임으로 다시 전환합니다.
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame' to find the confirm button")

                # "좌석선택완료" 버튼 찾기
                try:
                    # XPath를 수정하여 버튼 찾기
                    confirm_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
                    )
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchWindowException as e:
            print(f"No such window exception occurred: {e}")
            print("Available window handles:", driver.window_handles)
            traceback.print_exc()
            return False  # 창이 닫혔으므로 함수 종료
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()
            print("Available window handles:", driver.window_handles)

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

In [8]:
# 성공

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json
import time
import traceback
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException, TimeoutException


# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용) 
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용) 
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")


def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임으로 전환
            iframe_seat = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
            )
            driver.switch_to.frame(iframe_seat)
            print("Switched to iframe 'ifrmSeatFrame'")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 잠시 대기
                time.sleep(1)

                # 알림 창 처리 (필요한 경우)
                try:
                    WebDriverWait(driver, 3).until(EC.alert_is_present())
                    alert = driver.switch_to.alert
                    print(f"Alert text: {alert.text}")
                    alert.accept()
                    print("Alert accepted.")
                except TimeoutException:
                    print("No alert appeared after seat selection.")

                # 좌석 선택 후 기본 컨텐츠로 전환
                driver.switch_to.default_content()
                print("Switched to default content after clicking the seat")

                # 필요한 경우, 다시 프레임 전환
                # 여기서는 'ifrmSeatFrame' 프레임으로 다시 전환합니다.
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame' to find the confirm button")

                # "좌석선택완료" 버튼 찾기
                try:
                    # XPath를 수정하여 버튼 찾기
                    confirm_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
                    )
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchWindowException as e:
            print(f"No such window exception occurred: {e}")
            print("Available window handles:", driver.window_handles)
            traceback.print_exc()
            return False  # 창이 닫혔으므로 함수 종료
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()
            print("Available window handles:", driver.window_handles)

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")


쿠키 로드 성공
예매하기 버튼 클릭 완료
새 창으로 제어를 전환했습니다.
날짜 선택 완료
회차 선택 완료
좌석 선택 완료
Attempt 1 out of 100
Switched to default content
Switched to iframe 'ifrmSeatFrame'
8개의 선택 가능한 좌석을 찾았습니다.
좌석을 선택했습니다.
No alert appeared after seat selection.
Switched to default content after clicking the seat
Switched to iframe 'ifrmSeatFrame' to find the confirm button
좌석선택완료 버튼을 클릭했습니다.
좌석 선택에 성공했습니다.


In [ ]:
# 속도 향상상

In [ ]:
# 성공

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json
import time
import traceback
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException, TimeoutException


# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용) 
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용) 
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")


def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임으로 전환
            iframe_seat = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
            )
            driver.switch_to.frame(iframe_seat)
            print("Switched to iframe 'ifrmSeatFrame'")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 잠시 대기
                time.sleep(1)

                # 알림 창 처리 (필요한 경우)
                try:
                    WebDriverWait(driver, 3).until(EC.alert_is_present())
                    alert = driver.switch_to.alert
                    print(f"Alert text: {alert.text}")
                    alert.accept()
                    print("Alert accepted.")
                except TimeoutException:
                    print("No alert appeared after seat selection.")

                # 좌석 선택 후 기본 컨텐츠로 전환
                driver.switch_to.default_content()
                print("Switched to default content after clicking the seat")

                # 필요한 경우, 다시 프레임 전환
                # 여기서는 'ifrmSeatFrame' 프레임으로 다시 전환합니다.
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame' to find the confirm button")

                # "좌석선택완료" 버튼 찾기
                try:
                    # XPath를 수정하여 버튼 찾기
                    confirm_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
                    )
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchWindowException as e:
            print(f"No such window exception occurred: {e}")
            print("Available window handles:", driver.window_handles)
            traceback.print_exc()
            return False  # 창이 닫혔으므로 함수 종료
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()
            print("Available window handles:", driver.window_handles)

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")


In [ ]:
# 성공

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json
import time
import traceback
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException, TimeoutException


# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용) 
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용) 
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")


def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임으로 전환
            iframe_seat = driver.find_element(By.NAME, 'ifrmSeatFrame')
            driver.switch_to.frame(iframe_seat)
            print("Switched to iframe 'ifrmSeatFrame'")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 바로 버튼 클릭 시도
                try:
                    confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")


# '다음단계' 버튼 클릭 함수 정의
def click_next_step(times=1):
    for i in range(times):
        try:
            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print(f"Attempting to click '다음단계' 버튼 ({i+1}/{times})")

            # 페이지 로딩 대기 (필요한 경우)
            time.sleep(1)  # 또는 WebDriverWait를 사용하여 특정 요소가 로드될 때까지 기다립니다.

            # '다음단계' 버튼 찾기
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
            )
            # 부모 <a> 태그 클릭
            parent_element = next_button.find_element(By.XPATH, "..")
            parent_element.click()
            print(f"'다음단계' 버튼을 클릭했습니다. ({i+1}/{times})")

            # 페이지 로딩 대기 (필요한 경우)
            time.sleep(1)  # 필요에 따라 조정
        except Exception as e:
            print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
            traceback.print_exc()
            return False
    return True

In [ ]:
# 속도 향상 삭제 금지

In [ ]:
# JYP 버전

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 100
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame'")
            except TimeoutException:
                print("iframe 'ifrmSeatFrame'을 찾지 못했습니다. 다시 시도합니다.")
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 바로 버튼 클릭 시도
                try:
                    confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
# 트와이스 버전

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame'")
            except TimeoutException:
                print("iframe 'ifrmSeatFrame'을 찾지 못했습니다. 다시 시도합니다.")
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 바로 버튼 클릭 시도
                try:
                    confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
# 속도 증강

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
            except TimeoutException:
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석만 찾기 (XPath 수정)
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @class='s13']")

            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                for seat in seats:
                    try:
                        seat.click()
                        print("좌석을 선택했습니다.")

                        # 좌석 선택 확인 (알림 창 처리)
                        try:
                            WebDriverWait(driver, 1).until(EC.alert_is_present())
                            alert = driver.switch_to.alert
                            alert_text = alert.text
                            alert.accept()
                            print(f"알림 창이 나타났습니다: {alert_text}")
                            # 알림 창이 나타났다면 좌석 선택에 실패한 것이므로 다음 좌석으로 넘어갑니다.
                            continue
                        except TimeoutException:
                            pass  # 알림 창이 없으면 좌석 선택 성공

                        # 좌석 클릭 후 바로 버튼 클릭 시도
                        try:
                            confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                            confirm_button.click()
                            print("좌석선택완료 버튼을 클릭했습니다.")
                        except Exception as e:
                            print(f"버튼 클릭에 실패했습니다: {e}")
                            traceback.print_exc()
                            return False
                        return True  # 좌석 선택 및 확인 버튼 클릭 성공
                    except Exception as e:
                        print(f"좌석 선택에 실패했습니다: {e}")
                        continue  # 다음 좌석으로 넘어감
                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과


# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
# 선택 불가능한 영역 필터링하기

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    failed_seat_ids = []  # 오류가 발생한 좌석의 id를 저장할 리스트

    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
            except TimeoutException:
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석만 찾기 (XPath 수정)
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")

            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    if seat_id in failed_seat_ids:
                        print(f"이미 선택 실패한 좌석 {seat_id}를 건너뜁니다.")
                        continue  # 이미 실패한 좌석은 건너뜁니다.

                    try:
                        seat.click()
                        print(f"좌석 {seat_id}를 선택했습니다.")

                        # 좌석 선택 확인 (알림 창 처리)
                        try:
                            WebDriverWait(driver, 1).until(EC.alert_is_present())
                            alert = driver.switch_to.alert
                            alert_text = alert.text
                            alert.accept()
                            print(f"알림 창이 나타났습니다: {alert_text}")
                            # 알림 창이 나타났다면 좌석 선택에 실패한 것이므로 해당 좌석 id를 저장하고 다음 좌석으로 넘어갑니다.
                            failed_seat_ids.append(seat_id)
                            continue
                        except TimeoutException:
                            pass  # 알림 창이 없으면 좌석 선택 성공

                        # 좌석 클릭 후 바로 버튼 클릭 시도
                        try:
                            confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                            confirm_button.click()
                            print("좌석선택완료 버튼을 클릭했습니다.")
                        except Exception as e:
                            print(f"버튼 클릭에 실패했습니다: {e}")
                            traceback.print_exc()
                            return False
                        return True  # 좌석 선택 및 확인 버튼 클릭 성공
                    except Exception as e:
                        print(f"좌석 {seat_id} 선택에 실패했습니다: {e}")
                        # 선택 실패한 좌석 id를 저장
                        failed_seat_ids.append(seat_id)
                        continue  # 다음 좌석으로 넘어감
                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과



# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
# 필터링 정보 추가 + 좌석 정보 프린트

In [ ]:
# 트와이스 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]')) # //*[@id="imgEnglishBooking"] 
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    failed_seat_ids = []  # 오류가 발생한 좌석의 id를 저장할 리스트

    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                print("스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
            except TimeoutException:
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석만 찾기 (XPath 수정)
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")

            # 여기서 좌석의 개수를 출력합니다
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seats:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")
                print("좌석 목록:")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    seat_title = seat.get_attribute('title')
                    print(f"- ID: {seat_id}, 정보: {seat_title}")
                    if seat_id in failed_seat_ids:
                        print(f"이미 선택 실패한 좌석 {seat_id}를 건너뜁니다.")
                        continue  # 이미 실패한 좌석은 건너뜁니다.

                    try:
                        seat.click()
                        print(f"좌석 {seat_id}를 선택했습니다.")

                        # 좌석 선택 확인 (알림 창 처리)
                        try:
                            WebDriverWait(driver, 1).until(EC.alert_is_present())
                            alert = driver.switch_to.alert
                            alert_text = alert.text
                            alert.accept()
                            print(f"알림 창이 나타났습니다: {alert_text}")
                            # 알림 창이 나타났다면 좌석 선택에 실패한 것이므로 해당 좌석 id를 저장하고 다음 좌석으로 넘어갑니다.
                            failed_seat_ids.append(seat_id)
                            continue
                        except TimeoutException:
                            pass  # 알림 창이 없으면 좌석 선택 성공

                        # 좌석 클릭 후 바로 버튼 클릭 시도
                        try:
                            confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                            confirm_button.click()
                            print("좌석선택완료 버튼을 클릭했습니다.")
                        except Exception as e:
                            print(f"버튼 클릭에 실패했습니다: {e}")
                            traceback.print_exc()
                            return False
                        return True  # 좌석 선택 및 확인 버튼 클릭 성공
                    except Exception as e:
                        print(f"좌석 {seat_id} 선택에 실패했습니다: {e}")
                        # 선택 실패한 좌석 id를 저장
                        failed_seat_ids.append(seat_id)
                        continue  # 다음 좌석으로 넘어감
                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
    
    # 좌석 선택 후 Telegram으로 알림 전송
    token = '7616270521:AAGXLoX51E0H6iqbuBN9TfFq0WqoSphjK1c'

    chat_id = '7889718434'  # 확인된 Chat ID
    message = "좌석 선택이 완료되었습니다!"

else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [10]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    params = {
        'chat_id': chat_id,
        'text': message
    }
    response = requests.get(url, params=params)
    return response

# 좌석 선택이 완료된 후 알림 전송
token = '7616270521:AAGXLoX51E0H6iqbuBN9TfFq0WqoSphjK1c'
chat_id = '7889718434'

message = "좌석 선택이 완료되었습니다!"
send_telegram_message(token, chat_id, message)

<Response [401]>

In [ ]:
# jyp 버전 테스트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    failed_seat_ids = []  # 오류가 발생한 좌석의 id를 저장할 리스트

    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
            except TimeoutException:
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석만 찾기 (XPath 수정)
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")

            # 여기서 좌석의 개수를 출력합니다
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seats:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")
                print("좌석 목록:")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    seat_title = seat.get_attribute('title')
                    print(f"- ID: {seat_id}, 정보: {seat_title}")
                    if seat_id in failed_seat_ids:
                        print(f"이미 선택 실패한 좌석 {seat_id}를 건너뜁니다.")
                        continue  # 이미 실패한 좌석은 건너뜁니다.

                    try:
                        seat.click()
                        print(f"좌석 {seat_id}를 선택했습니다.")

                        # 좌석 선택 확인 (알림 창 처리)
                        try:
                            WebDriverWait(driver, 1).until(EC.alert_is_present())
                            alert = driver.switch_to.alert
                            alert_text = alert.text
                            alert.accept()
                            print(f"알림 창이 나타났습니다: {alert_text}")
                            # 알림 창이 나타났다면 좌석 선택에 실패한 것이므로 해당 좌석 id를 저장하고 다음 좌석으로 넘어갑니다.
                            failed_seat_ids.append(seat_id)
                            continue
                        except TimeoutException:
                            pass  # 알림 창이 없으면 좌석 선택 성공

                        # 좌석 클릭 후 바로 버튼 클릭 시도
                        try:
                            confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                            confirm_button.click()
                            print("좌석선택완료 버튼을 클릭했습니다.")
                        except Exception as e:
                            print(f"버튼 클릭에 실패했습니다: {e}")
                            traceback.print_exc()
                            return False
                        return True  # 좌석 선택 및 확인 버튼 클릭 성공
                    except Exception as e:
                        print(f"좌석 {seat_id} 선택에 실패했습니다: {e}")
                        # 선택 실패한 좌석 id를 저장
                        failed_seat_ids.append(seat_id)
                        continue  # 다음 좌석으로 넘어감
                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # //*[@id="imgEnglishBooking"]
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    failed_seat_ids = []  # 오류가 발생한 좌석의 id를 저장할 리스트

    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                print("스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
            except TimeoutException:
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석만 찾기 (XPath 수정)
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")

            # 여기서 좌석의 개수를 출력합니다
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seats:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")
                print("좌석 목록:")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    seat_title = seat.get_attribute('title')
                    print(f"- ID: {seat_id}, 정보: {seat_title}")
                    if seat_id in failed_seat_ids:
                        print(f"이미 선택 실패한 좌석 {seat_id}를 건너뜁니다.")
                        continue  # 이미 실패한 좌석은 건너뜁니다.

                    try:
                        seat.click()
                        print(f"좌석 {seat_id}를 선택했습니다.")

                        # 좌석 선택 확인 (알림 창 처리)
                        try:
                            WebDriverWait(driver, 1).until(EC.alert_is_present())
                            alert = driver.switch_to.alert
                            alert_text = alert.text
                            alert.accept()
                            print(f"알림 창이 나타났습니다: {alert_text}")
                            # 알림 창이 나타났다면 좌석 선택에 실패한 것이므로 해당 좌석 id를 저장하고 다음 좌석으로 넘어갑니다.
                            failed_seat_ids.append(seat_id)
                            continue
                        except TimeoutException:
                            pass  # 알림 창이 없으면 좌석 선택 성공

                        # 좌석 클릭 후 바로 버튼 클릭 시도
                        try:
                            confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                            confirm_button.click()
                            print("좌석선택완료 버튼을 클릭했습니다.")
                        except Exception as e:
                            print(f"버튼 클릭에 실패했습니다: {e}")
                            traceback.print_exc()
                            return False
                        return True  # 좌석 선택 및 확인 버튼 클릭 성공
                    except Exception as e:
                        print(f"좌석 {seat_id} 선택에 실패했습니다: {e}")
                        # 선택 실패한 좌석 id를 저장
                        failed_seat_ids.append(seat_id)
                        continue  # 다음 좌석으로 넘어감
                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")

    # '다음단계' 버튼 클릭 함수 정의
    def click_next_step():
        try:
            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Attempting to click '다음단계' 버튼")

            # '다음단계' 버튼이 로드될 때까지 기다림
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
            )
            # 부모 <a> 태그 클릭
            parent_element = next_button.find_element(By.XPATH, "..")
            parent_element.click()
            print("'다음단계' 버튼을 클릭했습니다.")
        except Exception as e:
            print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
            traceback.print_exc()
            return False
        return True

    # '다음단계' 버튼 클릭
    if click_next_step():
        print("다음 단계로 이동했습니다.")
    else:
        print("다음 단계로 이동하지 못했습니다.")

else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")

# 좌석 선택 함수
def select_seat_by_id(seat_id):
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()

        # 좌석 선택 프레임으로 전환
        iframe_seat = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
        )
        driver.switch_to.frame(iframe_seat)
        print("좌석 선택 프레임으로 전환했습니다.")

        # 해당 좌석 요소 찾기
        seat = driver.find_element(By.ID, seat_id)
        print(f"좌석 {seat_id}를 찾았습니다.")

        # 좌석 클릭 (JavaScript로 실행)
        driver.execute_script("arguments[0].click();", seat)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        time.sleep(0.5)  # 필요에 따라 시간 조절

        # 선택된 좌석이 실제로 등록되었는지 확인
        selected_seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'SeatSelected')]")
        if not selected_seats:
            print("좌석이 실제로 선택되지 않았습니다.")
            return False
        else:
            print("좌석이 성공적으로 선택되었습니다.")

        # 좌석선택완료 버튼 클릭
        confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")

        return True  # 좌석 선택 및 확인 버튼 클릭 성공

    except Exception as e:
        print(f"좌석 선택에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도
seat_id = 't4100023'  # 선택하려는 좌석 ID를 입력하세요
if select_seat_by_id(seat_id):
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
            return False
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

# 스크립트 종료 또는 브라우저 유지
# driver.quit()


쿠키 로드 성공
예매하기 버튼 클릭 완료
새 창으로 제어를 전환했습니다.
날짜 선택 완료
회차 선택 완료
좌석 선택 버튼 클릭 완료
좌석 선택 프레임으로 전환했습니다.
좌석 t4100023를 찾았습니다.
좌석 t4100023를 클릭했습니다.
좌석이 실제로 선택되지 않았습니다.
좌석 선택에 실패했습니다.
Attempting to click '다음단계' 버튼
'다음단계' 버튼 클릭에 실패했습니다: Message: 

다음 단계로 이동하지 못했습니다.


Traceback (most recent call last):
  File "C:\Users\withd\AppData\Local\Temp\ipykernel_14468\208549407.py", line 177, in click_next_step
    next_button = WebDriverWait(driver, 10).until(
  File "c:\Users\withd\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat_by_id(seat_id):
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()

        # 좌석 선택 프레임으로 전환
        iframe_seat = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
        )
        driver.switch_to.frame(iframe_seat)
        print("좌석 선택 프레임으로 전환했습니다.")

        # 해당 좌석 요소 찾기
        seat = driver.find_element(By.ID, seat_id)
        print(f"좌석 {seat_id}를 찾았습니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        time.sleep(0.5)  # 필요에 따라 시간 조절

        # 선택된 좌석이 실제로 등록되었는지 확인
        # 예시: 선택된 좌석의 클래스에 'selected'가 추가되었는지 확인
        selected = False
        try:
            # 좌석의 class 속성을 확인하여 선택되었는지 확인
            seat_class = seat.get_attribute('class')
            if 'selected' in seat_class.lower() or 'selected' in seat_class:
                selected = True
        except Exception as e:
            print(f"좌석 클래스 확인 오류: {e}")

        if not selected:
            print("좌석이 실제로 선택되지 않았습니다.")
            return False
        else:
            print("좌석이 성공적으로 선택되었습니다.")

        # 좌석선택완료 버튼 클릭
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")

        return True  # 좌석 선택 및 확인 버튼 클릭 성공

    except NoSuchElementException:
        print(f"좌석 {seat_id} 요소를 찾을 수 없습니다.")
        return False
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석 선택에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat(seat_id):
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 좌석 선택 시도
            success = select_seat_by_id(seat_id)
            if success:
                print("좌석 선택에 성공했습니다.")
                return True
            else:
                print("좌석 선택에 실패했습니다. 다시 시도합니다.")
        except Exception as e:
            print(f"모니터링 중 예기치 않은 오류 발생: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도
seat_id = 't4100023'  # 선택하려는 좌석 ID를 입력하세요
if monitor_and_select_seat(seat_id):
    print("좌석 선택에 성공했습니다.")
    # '다음단계' 버튼 클릭
    if click_next_step():
        print("다음 단계로 이동했습니다.")
    else:
        print("다음 단계로 이동하지 못했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# 브라우저 유지 또는 종료
# 필요에 따라 주석을 해제하세요
# driver.quit()


In [ ]:
# 페이지 새로 고침 추가 - 현재 오류

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

time.sleep(5)

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        time.sleep(0.5)  # 필요에 따라 시간 조절

        # 선택된 좌석이 실제로 등록되었는지 확인
        # 실제 웹사이트의 선택된 좌석 확인 로직에 맞게 수정 필요
        # 예시: 선택된 좌석에 'selected' 클래스가 추가되었는지 확인
        seat_class = seat_element.get_attribute('class')
        if 'selected' in seat_class.lower() or 'selected-seat' in seat_class.lower():
            print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
            return True
        else:
            print(f"좌석 {seat_id} 선택 실패.")
            return False

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 페이지 새로 고침 및 좌석 선택 프레임 로드 대기 추가
            driver.refresh()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
            )

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()

            # 좌석 선택 프레임으로 전환
            try:
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("좌석 선택 프레임으로 전환했습니다.")
            except TimeoutException:
                print("좌석 선택 프레임을 찾을 수 없습니다. 다시 시도합니다.")
                time.sleep(monitoring_interval)
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")
                print("좌석 목록:")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    seat_title = seat.get_attribute('title')
                    print(f"- ID: {seat_id}, 정보: {seat_title}")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

time.sleep(5)

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        time.sleep(0.5)  # 필요에 따라 시간 조절

        # 선택된 좌석이 실제로 등록되었는지 확인
        # 실제 웹사이트의 선택된 좌석 확인 로직에 맞게 수정 필요
        # 예시: 선택된 좌석에 'selected' 클래스가 추가되었는지 확인
        seat_class = seat_element.get_attribute('class')
        if 'selected' in seat_class.lower() or 'selected-seat' in seat_class.lower():
            print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
            return True
        else:
            print(f"좌석 {seat_id} 선택 실패.")
            return False

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 좌석 선택 프레임만 새로 고침
            try:
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("좌석 선택 프레임으로 전환했습니다.")

                # 프레임 내에서 새로 고침
                driver.execute_script("location.reload();")
                print("좌석 선택 프레임을 새로 고쳤습니다.")

                # 프레임이 다시 로드될 때까지 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                print("좌석 선택 프레임이 다시 로드되었습니다.")
            except TimeoutException:
                print("좌석 선택 프레임을 새로 고치는 데 실패했습니다. 다시 시도합니다.")
                driver.switch_to.default_content()
                time.sleep(monitoring_interval)
                continue  # 다음 시도로 넘어감

            # 좌석 선택 프레임으로 다시 전환
            driver.switch_to.frame('ifrmSeatFrame')

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")
                print("좌석 목록:")
                for seat in seats:
                    seat_id = seat.get_attribute('id')
                    seat_title = seat.get_attribute('title')
                    print(f"- ID: {seat_id}, 정보: {seat_title}")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False


# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()

In [ ]:
# 느려서 아래껄로

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        WebDriverWait(driver, 1).until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 좌석 선택 프레임으로 전환 (이미 전환했으므로 생략)
            # driver.switch_to.frame(iframe_seat)

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()
# 삭제 금지

In [ ]:
# 글로벌 시도

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imgEnglishBooking"]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        WebDriverWait(driver, 1).until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 좌석 선택 프레임으로 전환 (이미 전환했으므로 생략)
            # driver.switch_to.frame(iframe_seat)

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()
# 삭제 금지

In [ ]:
# 글로벌 시도

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imgEnglishBooking"]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# **6. 새로운 탭에서 두 번째 booking 버튼 클릭**
def click_second_booking():
    try:
        # 두 번째 booking 버튼이 로드될 때까지 대기
        booking_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="hlkPurchase"]'))  # 두 번째 booking 버튼 XPath
        )
        booking_button.click()
        print("두 번째 booking 버튼 클릭 완료")
    except Exception as e:
        print(f"두 번째 booking 버튼 클릭 오류: {e}")
        driver.quit()
        exit()

# 두 번째 booking 버튼 클릭 호출
click_second_booking()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        WebDriverWait(driver, 1).until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()


In [ ]:
# booking 쿠기 추가하기 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51057')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
# 쿠키 로드
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:  # UTF-8 인코딩 지정
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# Booking 버튼 찾기 (ID로 찾기)
booking_button = driver.find_element(By.ID, 'hlkPurchase')

# 버튼 클릭
booking_button.click()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except Exception as e:
    print(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        WebDriverWait(driver, 1).until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"좌석선택완료 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    main()


In [ ]:
# 글로벌 도전 성공, 삭제 금지

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import json
import traceback
import datetime
import time

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
MAX_ATTEMPTS = 100000
MONITORING_INTERVAL = 0.5  # 초
WAIT_TIME = 10  # 기본 대기 시간 (초)

# WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51057')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            # 'sameSite' 속성 제거 (필요 시)
            cookie.pop('sameSite', None)
            # 쿠키에 필요한 도메인 정보가 있는지 확인
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드되고 특정 요소가 나타날 때까지 대기
try:
    # 예: 'hlkPurchase' 버튼이 클릭 가능할 때까지 대기
    wait.until(EC.element_to_be_clickable((By.ID, 'hlkPurchase')))
    print("페이지 로드 완료")
except TimeoutException:
    print("페이지 로드 대기 시간 초과")
    driver.quit()
    exit()

# Booking 버튼 클릭
try:
    booking_button = driver.find_element(By.ID, 'hlkPurchase')
    booking_button.click()
    print("Booking 버튼 클릭 완료")
except NoSuchElementException:
    print("Booking 버튼을 찾을 수 없습니다.")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 새 창으로 제어 전환
try:
    wait.until(EC.number_of_windows_to_be(2))  # 새 창이 열릴 때까지 대기
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except TimeoutException:
    print("새 창이 열리지 않았습니다.")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    # 날짜 버튼이 로드되고 클릭 가능할 때까지 대기
    date_button = wait.until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 실제 날짜 ID로 수정 필요
    )
    date_button.click()
    print("날짜 선택 완료")
except TimeoutException:
    print("날짜 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except TimeoutException:
    print("회차 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = wait.until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except TimeoutException:
    print("좌석 선택 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        wait_short = WebDriverWait(driver, 2)
        wait_short.until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'좌석선택완료' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'좌석선택완료' 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0

    while attempts < MAX_ATTEMPTS:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {MAX_ATTEMPTS}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(MONITORING_INTERVAL)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드되고 클릭 가능할 때까지 대기
        next_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'다음단계' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"스크립트 실행 중 예기치 않은 오류 발생: {e}")
        traceback.print_exc()
    finally:
        # 모든 작업 후 브라우저를 종료하려면 아래 주석을 해제하세요
        # driver.quit()
        pass


WebDriverException: Message: target frame detached: received Inspector.detached event
  (failed to check if window was closed: disconnected: unable to send message to renderer)
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00007FF6092CB632+29090]
	(No symbol) [0x00007FF60923E6E9]
	(No symbol) [0x00007FF6090FB1CA]
	(No symbol) [0x00007FF6090E56A7]
	(No symbol) [0x00007FF6090E4486]
	(No symbol) [0x00007FF6090E3CD1]
	(No symbol) [0x00007FF6090E3C10]
	(No symbol) [0x00007FF6090E2AFB]
	(No symbol) [0x00007FF609109166]
	(No symbol) [0x00007FF60919693E]
	(No symbol) [0x00007FF609176493]
	(No symbol) [0x00007FF6091409B1]
	(No symbol) [0x00007FF609141B11]
	GetHandleVerifier [0x00007FF6095E881D+3294093]
	GetHandleVerifier [0x00007FF609634403+3604339]
	GetHandleVerifier [0x00007FF60962A2C7+3563063]
	GetHandleVerifier [0x00007FF609386F16+797318]
	(No symbol) [0x00007FF60924986F]
	(No symbol) [0x00007FF609245454]
	(No symbol) [0x00007FF6092455E0]
	(No symbol) [0x00007FF6

In [23]:
# 테스트 용 다른팀
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# WebDriverWait 인스턴스 생성
WAIT_TIME = 10  # 기본 대기 시간 (초)
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51131')

# 2. 저장된 쿠키 불러오기
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            cookie.pop('sameSite', None)
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 잔여 좌석 모니터링 함수
def monitor_remaining_seats():
    try:
        # 아이프레임으로 전환
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "ifrmSeatFrame")))
        print("iframe으로 전환 성공")

        # 잔여 좌석을 모니터링
        seat_info_xpath = "//li[@id='liLegend']/div/p/em"
        
        while True:
            # 좌석 정보를 찾고 텍스트 추출
            remaining_seats = driver.find_element(By.XPATH, seat_info_xpath).text
            print(f"잔여 좌석 정보: {remaining_seats}")
            
            # 남은 좌석이 0이 아닐 때 알림 표시
            if remaining_seats != "0 seat(s)":
                print(f"잔여 좌석이 있습니다: {remaining_seats}")
                break

            # 5초 간격으로 확인
            time.sleep(5)

    except Exception as e:
        print(f"좌석 모니터링 중 오류 발생: {e}")

# 모니터링 시작
monitor_remaining_seats()

쿠키 로드 성공
좌석 모니터링 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.90)
Stacktrace:
	GetHandleVerifier [0x00007FF63656B632+29090]
	(No symbol) [0x00007FF6364DE6E9]
	(No symbol) [0x00007FF63639B1CA]
	(No symbol) [0x00007FF63636FAF5]
	(No symbol) [0x00007FF63641E2E7]
	(No symbol) [0x00007FF636435EE1]
	(No symbol) [0x00007FF636416493]
	(No symbol) [0x00007FF6363E09B1]
	(No symbol) [0x00007FF6363E1B11]
	GetHandleVerifier [0x00007FF63688881D+3294093]
	GetHandleVerifier [0x00007FF6368D4403+3604339]
	GetHandleVerifier [0x00007FF6368CA2C7+3563063]
	GetHandleVerifier [0x00007FF636626F16+797318]
	(No symbol) [0x00007FF6364E986F]
	(No symbol) [0x00007FF6364E5454]
	(No symbol) [0x00007FF6364E55E0]
	(No symbol) [0x00007FF6364D4A7F]
	BaseThreadInitThunk [0x00007FF89441257D+29]
	RtlUserThreadStart [0x00007FF89562AF28+40]



In [ ]:
# 다음 이동 부분 수정 (글로벌에 맞게)

In [20]:
# 다른 아티스트 테스트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import json
import traceback
import datetime

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
MAX_ATTEMPTS = 10000
MONITORING_INTERVAL = 0.5  # 초
WAIT_TIME = 10  # 기본 대기 시간 (초)

# WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51131')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            cookie.pop('sameSite', None)
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드되고 특정 요소가 나타날 때까지 대기
try:
    wait.until(EC.element_to_be_clickable((By.ID, 'hlkPurchase')))
    print("페이지 로드 완료")
except TimeoutException:
    print("페이지 로드 대기 시간 초과")
    driver.quit()
    exit()

# Booking 버튼 클릭
try:
    booking_button = driver.find_element(By.ID, 'hlkPurchase')
    booking_button.click()
    print("Booking 버튼 클릭 완료")
except NoSuchElementException:
    print("Booking 버튼을 찾을 수 없습니다.")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 새 창으로 제어 전환
try:
    wait.until(EC.number_of_windows_to_be(2))
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except TimeoutException:
    print("새 창이 열리지 않았습니다.")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    date_button = wait.until(EC.element_to_be_clickable((By.ID, "2024-11-30")))
    date_button.click()
    print("날짜 선택 완료")
except TimeoutException:
    print("날짜 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]')))
    time_button.click()
    print("회차 선택 완료")
except TimeoutException:
    print("회차 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnSeatSelect')))
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except TimeoutException:
    print("좌석 선택 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

def switch_to_seat_iframe():
    try:
        seat_iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "ifrmSeatFrame")))
        print("iframe으로 전환 성공")
    except Exception as e:
        print(f"iframe으로 전환 실패: {e}")
        traceback.print_exc()

# 좌석 선택 및 검증 함수
def select_seat_and_move_next(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석선택완료 버튼 클릭
        confirm_success = click_confirm()
        if confirm_success:
            print("좌석 선택 완료 및 다음 단계로 이동 성공.")
            return True
        else:
            print("좌석 선택 완료 버튼 클릭 실패. 다시 시도합니다.")
            return False
    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']")))
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except Exception as e:
        print(f"'좌석선택완료' 버튼 클릭에 실패했습니다: {e}")
        return False

# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat():
    attempts = 0

    switch_to_seat_iframe()

    while attempts < MAX_ATTEMPTS:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {MAX_ATTEMPTS}")

            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade='VIP석']")

            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                for seat in seats:
                    success = select_seat_and_move_next(seat)
                    if success:
                        return True  # 좌석 선택 및 다음 단계로 이동 성공
                    else:
                        continue

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        time.sleep(MONITORING_INTERVAL)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수 (첫 번째 버튼만)
def click_next_step():
    try:
        driver.switch_to.default_content()  # 기본 컨텐츠로 전환

        # '다음단계' 버튼이 화면에 보이고, 상호작용 가능할 때까지 대기
        next_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//img[contains(@src, 'btn_next02.gif')]")))
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//img[contains(@src, 'btn_next02.gif')]")))
        next_button.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        return False

if __name__ == "__main__":
    try:
        monitor_and_select_seat()
    except Exception as e:
        print(f"스크립트 실행 중 예기치 않은 오류 발생: {e}")
        traceback.print_exc()
    finally:
        pass


쿠키 로드 성공
페이지 로드 완료
Booking 버튼 클릭 완료
새 창으로 제어를 전환했습니다.
날짜 선택 완료
회차 선택 완료
좌석 선택 버튼 클릭 완료
iframe으로 전환 성공
Attempt 1 out of 10000
조건을 만족하는 좌석은 총 52개입니다.
52개의 선택 가능한 좌석을 찾았습니다.
좌석 t1400019 (Standing Admission number 090 Col)를 선택 시도합니다.
좌석 t1400019를 클릭했습니다.
좌석선택완료 버튼을 클릭했습니다.
좌석 선택 완료 및 다음 단계로 이동 성공.
